In [1]:
cd ..

/data/cphillips/isccp-ng/isccp_l1g_prototype


In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import xarray as xr
import numpy as np
import pandas as pd

In [4]:
import make_gif

In [14]:
from pathlib import Path

In [5]:
import PIL.Image

In [6]:
from tqdm import tqdm

In [8]:
f = 'abi_scanline_time_luts/ABI-Time_Model_LUTS/ABI-Timeline05B_Mode 6A_20190612-183017.nc'

In [8]:
f = 'abi_scanline_time_luts/ABI-Time_Model_LUTS/A'

In [42]:
ds.CONUS1_pixel_times

<xarray.DataArray 'CONUS1_pixel_times' (CONUS1_cols: 2500, CONUS1_rows: 1500)>
[3750000 values with dtype=timedelta64[ns]]
Dimensions without coordinates: CONUS1_cols, CONUS1_rows
Attributes:
    coordinates:  (row, column)
    comment:      Time relative to first product pixel

In [40]:
ds.FD_pixel_times

<xarray.DataArray 'FD_pixel_times' (FD_cols: 5424, FD_rows: 5424)>
[29419776 values with dtype=timedelta64[ns]]
Dimensions without coordinates: FD_cols, FD_rows
Attributes:
    coordinates:  (row, column)
    comment:      Time relative to first product pixel

In [9]:
ds = xr.open_dataset(f)

In [10]:
ds2 = xr.open_dataset('l1b/20201001T0000/g16/14/OR_ABI-L1b-RadF-M6C14_G16_s20202750000195_e20202750009503_c20202750009581.nc')

In [11]:
rad = (ds2.Rad[::4,::4].values/150).clip(0,1)

In [12]:
times = (ds.FD_pixel_times[::4,::4]/1e9).T.astype(np.float32).values

In [13]:
cmap = matplotlib.cm.get_cmap('Greys')

In [35]:
frames = np.arange(times.min(), times.max(), .5)
buffer = np.zeros((*rad.shape, 3), dtype=np.uint8)
video = Path('timeline.mp4').absolute()
with make_gif.make_video(video, framerate=24) as vid:
    with tqdm(frames[:]) as bar:
        for t in bar:
            buffer[:] = cmap(rad, bytes=True)[:,:,:3]
            buffer[times > t,:] = 0
            im = PIL.Image.fromarray(buffer)
            im.resize((1024,1024)).save(vid, format='png')

100%|██████████| 1152/1152 [04:48<00:00,  4.00it/s]


In [36]:
make_gif.play_video(Path('../timeline.mp4'))